In [1]:
# save new density

# repeat from step 2
# propagate density with velocity verlet

### generate start density
- run PROFESS for one step
- print density -> initial guess
- see /home/misa/git_repositories/PROFESS/test/single_iteration

### call profess-1
- copy intial density
- call profess-1
- do it from python

### propagate density
- load density and gradient into python
- reshape
- propagate (with velocity verlet)

In [1]:
import numpy as np

# load density
filep = '/home/misa/software/PROFESS-1/calculate_gradient/density'
with open(filep, 'r') as f:
    file = f.readlines()
density_unparsed = file[0][108:]
density = density_unparsed.split()
density = np.array([float(i) for i in density])
density = density.reshape((16,16,16))

In [3]:
# load gradient
filep = '/home/misa/software/PROFESS-1/calculate_gradient/dEdX'
dEdX = []
with open(filep, 'r') as f:
    for i, line in enumerate(f):
        if i != 0:
            dEdX.append(float(line.strip('\n')))
dEdX = np.array(dEdX)
dEdX = dEdX.reshape((16, 16, 16))

In [23]:
# what are the units of density and dE/dX?
import ase.units as au
V = (3.9691/au.Bohr)**3
dV = V/16**3

density_vox = density*dV
dEdX_vox = dEdX*dV

In [22]:
print((density*dV).sum())
print("Density must be converted to Voxel/Bohr^3")

11.999999866902229
Density must be converted to Voxel/Bohr^3


#### What are appropiate values for mu and dt?
- $\mu = 1$ and $dt = 0.1$ (both in a.u.) (10.1103/physrevlett.55.2471)

In [26]:
# propagate density
mu = 1
dt = 0.1
a = -dEdX_vox/mu
x_1 = a*dt**2 + np.sqrt(density_vox)

### enforce constraint

In [32]:
Ne = 12.0
N_p = (x_1*np.sqrt(density_vox)).sum()
N_pp = (x_1*x_1).sum()
tau = dt**2/mu

In [33]:
p2 = N_p/(tau*Ne)
q = (N_pp - Ne)/(tau**2*Ne)
lambda_2 = -p2 + np.sqrt(p2**2 - q)

In [85]:
x_1_constr = x_1 + dt**2/mu*lambda_2*x_1
density_1 = np.power(x_1_constr,2)

### save new density

In [106]:
start_str = '  x-dimension:          16   y-dimension:          16   z-dimension:          16   # of spins:            1 '
density_1_str = np.array_str(density_1.flatten(), precision=22)
density_1_str = density_1_str.replace('[', '')
density_1_str = density_1_str.replace(']', '')
# density_1_str = start_str + density_1_str
new_dens = '/home/misa/software/PROFESS-1/calculate_gradient/new_density'
with open(new_dens, 'w') as f:
    f.write(density_1_str)

In [107]:
len(density_1_str)

136

In [108]:
density_1_str

'0.0006178628409271254 0.0007710186073295659 0.0017288045802888257 ...\n 0.0033594210909632916 0.002433282711534812  0.0013635308119290886'